In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

# Replace these placeholders with your own values
owner = "BigDataIA-Spring2023-Team-04"
repo = "BigDataIA-Assignment-03"

api_token = os.environ.get("GITHUB_API")

# Set up the API request headers
headers = {
    "Authorization": f"token {api_token}",
    "Accept": "application/vnd.github+json",
}

page = 1
per_page = 100  # Maximum number of items per page allowed by the GitHub API

while True:
    # Set up the API request URL with pagination parameters
    url = f"https://api.github.com/repos/{owner}/{repo}/issues?page={page}&per_page={per_page}"

    # Fetch the list of issues for the current page
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        issues = response.json()

        if (
            not issues
        ):  # If the list of issues is empty, we've reached the end of the pages
            break

        for issue in issues:
            if "pull_request" not in issue:  # Filter out pull requests
                print(f"Issue #{issue['number']}: {issue['title']}")

        # Move on to the next page
        page += 1
    else:
        print(f"Error {response.status_code}: {response.text}")
        break

Issue #6: Directory Structure: A proper directory structure can be maintained 
Issue #5: Forget Password API : API is allowing the user to change anyone's password if they know their username 
Issue #4: code Issue:  password field not required for forgot password feature
Issue #3: Testing Goes or Nexrad links one by one
Issue #2: Same error for all incorrect CLI commands
Issue #1: Typer CLI requesting authentication on every command


In [5]:
def get_issue_details(owner, repo, issue_number, api_token):
    issue_url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"
    comments_url = (
        f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/comments"
    )
    reactions_url = (
        f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/reactions"
    )
    timeline_url = (
        f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/timeline"
    )

    headers = {
        "Authorization": f"token {api_token}",
        "Accept": "application/vnd.github+json",
    }

    reactions_headers = {
        "Authorization": f"token {api_token}",
        "Accept": "application/vnd.github.squirrel-girl-preview+json",
    }

    timeline_headers = {
        "Authorization": f"token {api_token}",
        "Accept": "application/vnd.github.mockingbird-preview+json",
    }

    issue_response = requests.get(issue_url, headers=headers)
    comments_response = requests.get(comments_url, headers=headers)
    reactions_response = requests.get(reactions_url, headers=reactions_headers)
    timeline_response = requests.get(timeline_url, headers=timeline_headers)

    if (
        issue_response.status_code == 200
        and comments_response.status_code == 200
        and reactions_response.status_code == 200
        and timeline_response.status_code == 200
    ):
        issue_data = issue_response.json()
        comments_data = comments_response.json()
        reactions_data = reactions_response.json()
        timeline_data = timeline_response.json()

        # Add comment ID and reactions to the comments_data
        for comment in comments_data:
            comment_id = comment["id"]
            comment_reactions_url = comment["reactions"]["url"]
            comment_reactions_response = requests.get(
                comment_reactions_url, headers=reactions_headers
            )
            if comment_reactions_response.status_code == 200:
                comment_reactions_data = comment_reactions_response.json()
                comment["comment_id"] = comment_id
                comment["reactions"] = comment_reactions_data
            else:
                print(
                    f"Error fetching comment reactions: {comment_reactions_response.status_code}"
                )

        issue_data["comments"] = comments_data
        issue_data["reactions"] = reactions_data
        issue_data["timeline"] = timeline_data
        return issue_data
    else:
        print(
            f"Error fetching issue details: {issue_response.status_code}, {comments_response.status_code}, {reactions_response.status_code}, {timeline_response.status_code}"
        )
        return None


# Replace these placeholders with your own values
owner = "BigDataIA-Spring2023-Team-04"
repo = "BigDataIA-Assignment-03"
issue_number = 6
api_token = os.environ.get("GITHUB_API")

issue_details = get_issue_details(owner, repo, issue_number, api_token)

if issue_details:
    print(f"Issue #{issue_details['number']}: {issue_details['title']}")
    print(f"Issue Created by {issue_details['user']['login']}")
    print(
        f"Assignees: {[assignee['login'] for assignee in issue_details['assignees']]}"
    )
    print(f"Labels: {[label['name'] for label in issue_details['labels']]}")
    print(
        f"Milestone: {issue_details['milestone']['title'] if issue_details['milestone'] else 'None'}"
    )
    print(f"Body: {issue_details['body']}")

    print("Reactions:")
    for reaction in issue_details["reactions"]:
        print(f"- {reaction['user']['login']} reacted with {reaction['content']}")

    print("Comments:")
    for comment in issue_details["comments"]:
        print(
            f"- {comment['user']['login']} (ID: {comment['comment_id']}): {comment['body']}"
        )
        print("  Reactions:")
        for reaction in comment["reactions"]:
            print(f"  - {reaction['user']['login']} reacted with {reaction['content']}")

    print("Timeline events:")
    for event in issue_details["timeline"]:
        if event["event"] == "cross-referenced":
            print(
                f"- {event['actor']['login']} cross-referenced this issue on {event['source']['issue']['html_url']}"
            )
        elif event["event"] == "added_to_project":
            print(f"- {event['actor']['login']} added this issue to a project")
        elif event["event"] == "milestoned":
            print(
                f"- {event['actor']['login']} added this issue to milestone {event['milestone']['title']}"
            )
        elif event["event"] == "assigned":
            print(
                f"- {event['actor']['login']} assigned {event['assignee']['login']} to this issue"
            )
        elif event["event"] == "unassigned":
            print(
                f"- {event['actor']['login']} unassigned {event['assignee']['login']} to this issue"
            )
        elif event["event"] == "labeled":
            print(
                f"- {event['actor']['login']} added the label '{event['label']['name']}' to this issue"
            )
        elif event["event"] == "referenced":
            print(f"- {event['actor']['login']} referenced this issue from a commit")

Issue #6: Directory Structure: A proper directory structure can be maintained 
Issue Created by hindupurv
Assignees: ['KaranAgrawal1509', 'Rizba11']
Labels: ['enhancement']
Milestone: Done
Body: <img width="1430" alt="Screenshot 2023-03-17 at 4 39 40 PM" src="https://user-images.githubusercontent.com/114629181/226044692-5bdc443c-9f92-45de-b26e-30b5f4208154.png">

Expected Behavior:
A simple directory structure to refer to codes easily and remember which piece of code is where without much hazzle.

Current Behavior:
The directory is not properly structured as finding the right files makes the application easier for everyone to use.

Possible Solution:
Restructuring the directory with appropriate file names and folder names.

Steps to Reproduce:
As shown in the image, you can make the changes to existing files and directories.

Context (Environment):
Code

Detailed Description:
An easy to understand directory structure allows easy testing and implementation of different applications.

Po

In [4]:
issue_details["timeline"]

[{'id': 8965613105,
  'node_id': 'AE_lADOJE80kM5hJqdGzwAAAAIWZGYx',
  'url': 'https://api.github.com/repos/BigDataIA-Spring2023-Team-04/BigDataIA-Assignment-03/issues/events/8965613105',
  'actor': {'login': 'KaranAgrawal1509',
   'id': 90572559,
   'node_id': 'MDQ6VXNlcjkwNTcyNTU5',
   'avatar_url': 'https://avatars.githubusercontent.com/u/90572559?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/KaranAgrawal1509',
   'html_url': 'https://github.com/KaranAgrawal1509',
   'followers_url': 'https://api.github.com/users/KaranAgrawal1509/followers',
   'following_url': 'https://api.github.com/users/KaranAgrawal1509/following{/other_user}',
   'gists_url': 'https://api.github.com/users/KaranAgrawal1509/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/KaranAgrawal1509/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/KaranAgrawal1509/subscriptions',
   'organizations_url': 'https://api.github.com/users/KaranAgrawal1509/orgs',
